In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install sentence-transformers

In [9]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append("./modules")
from embedding_loader import build_index

build_index("data/medical_snippets.csv")

from local_retriever import LocalRetriever
from web_retriever import WebRetriever
from triage_module import TriageClassifier
from fusion_ranker import FusionRanker
from generator import AnswerGenerator

#Setup
triage = TriageClassifier()
local = LocalRetriever()
web = WebRetriever(api_key="d592952404215ede63067becb052ef3efdd27310")
ranker = FusionRanker()
generator = AnswerGenerator()

def print_response_clean(query, generator, triage, local, web, ranker):
    condition, _ = triage.classify(query)
    local_docs = local.retrieve(query)
    web_docs = web.search(query)
    fused_docs = ranker.fuse(query, local_docs, web_docs)
    
    response = generator.generate(query, condition, fused_docs, web=None)

    # Remove disclaimer manually (if it exists)
    if response.startswith("⚠️"):
        response = response.split("\n", 1)[-1]  # remove first line only

    # Force split using periods and line breaks
    response_lines = response.replace("**", "").replace("\n", ". ").split(". ")
    
    print("\n--- Response ---\n")
    for line in response_lines:
        line = line.strip()
        if line and not line.startswith("Citations:"):
            print(line)
            
query = input("Enter your symptoms: ")
print_response_clean(query, generator, triage, local, web, ranker)

    
  

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Enter your symptoms:  I'm sweating, shaky, and my glucometer reads 55 mg/dL—what should I do right now?



--- Response ---

⚠️ This information is for educational purposes only and is not a substitute for professional medical advice
Likely Condition: Hypoglycemia First-Aid Steps: - For severe hypoglycaemia with unconsciousness, give intramuscular glucagon 1 mg if available
- Hypoglycaemia is defined as blood glucose < 70 mg/dL and needs rapid glucose intake
- First-aid for mild hypoglycaemia: give 15 g of fast-acting carbohydrate such as glucose tablets
- Hyperkalaemia (> 5.5 mmol/L) in CKD may require calcium gluconate and insulin–glucose infusion
- A fasting plasma glucose ≥ 126 mg/dL on two occasions confirms diabetes
Key Medicines: glucose, glucagon, insulin...
